In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data/keyworded.csv').drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [ ]:
df.head()

,heads,descs,keywords
0,inclement weather prevents liar from getting t...,PROVIDENCE RI—In spite of his best efforts to ...,"{'carlson': 0.25, 'weather': 0.196, 'spotty': ..."
1,mother comes pretty close to using word stream...,PATERSON NJ—Family sources told reporters Tues...,"{'burkhart': 0.356, 'she': 0.283, 'close': 0.2..."
2,richard bransons globalwarming donation nearly...,LONDON—Analysts are predicting that the 3 bill...,"{'branson': 0.439, 'virgin': 0.235, 'balloonba..."
3,shadow government getting too large to meet in...,COLUMBUS OH—With its membership swelling in re...,"{'marriotts': 0.177, 'meeting': 0.176, 'confer..."
4,ford develops new suv that runs purely on gaso...,DEARBORN MI—The Ford Motor Company announced W...,"{'gasoline': 0.354, 'petrola': 0.296, 'nair': ..."


In [ ]:
df = df.sample(n=10000)

In [ ]:
df_X = df[['descs']]
df_y = df[['keywords']]

In [ ]:
for i in df_y.index:
    df_y.at[i, 'heads'] = ['<startseq>'] + eval(df_y['heads'][i]) + ['<endseq>']

## Remove Infrequent Words

### X

In [ ]:
word_list = set()
word_sets = [set(eval(l)) for l in df_X['lemma_tokens'].values]
for s in word_sets:
    word_list = word_list.union(s)

In [ ]:
print('number of words before:', len(word_list))

number of words before: 87057


In [ ]:
word_dic = {i : 0 for i in word_list}

In [ ]:
sentences_X = []
for i in range(len(df_X['lemma_tokens'])):
    words_X = eval(df_X['lemma_tokens'].values[i])
    sentences_X.append(words_X)
    for j in range(len(words_X)):
        word_dic[words_X[j]] += 1

In [ ]:
word_threshold = 3

new_sentences_X = []
for i, sentence_X in enumerate(sentences_X):
    new_sentences_X.append([word for word in sentence_X if word_dic[word] >= word_threshold])

In [ ]:
print('number of words after:', len(set([word for sentence in new_sentences_X for word in sentence])))

number of words after: 33157


In [ ]:
sentences_X = new_sentences_X

In [ ]:
sum([len(sentence_X) for sentence_X in sentences_X])/len(sentences_X)

245.4086

### y

In [ ]:
sentences_y = df_y['tokens.1'].values

In [ ]:
word_list = set([word for sentence in sentences_y for word in sentence])

In [ ]:
print('number of words before:', len(word_list))

number of words after: 16824


In [ ]:
word_dic = {i: 0 for i in word_list}

In [ ]:
for sentence_y in sentences_y:
    for word in sentence_y:
        word_dic[word] += 1

In [ ]:
word_threshold = 3

new_sentences_y = []
for i, sentence_y in enumerate(sentences_y):
    new_sentences_y.append([word for word in sentence_y if word_dic[word] >= word_threshold])

In [ ]:
print('number of words after:', len(set([word for sentence in new_sentences_y for word in sentence])))

number of words after: 5409


In [ ]:
sentences_y = new_sentences_y

In [ ]:
max([len(sentence_y) for sentence_y in sentences_y])

31

## Save the New Data

In [ ]:
df['lemma_tokens'] = sentences_X

In [ ]:
df['tokens.1'] = sentences_y

In [ ]:
df.to_csv('data/testing.csv')

## Word2Vec

In [ ]:
from gensim.models import word2vec

w2v = word2vec.Word2Vec
body_embedding = w2v(sentences_X, min_count=1, size=100)
head_embedding = w2v(sentences_y, min_count=1, size=100)

In [ ]:
body_embedding.save('models/body_embedding.model')
head_embedding.save('models/head_embedding.model')

In [ ]:
vectors_body = []
vectors_head = []

In [ ]:
for i in range(len(sentences_X)):
    vectors_body.append([])
    temp = sentences_X[i]
    for j in range(len(temp)):
        vectors_body[i].append(body_embedding[temp[j]])

D:\Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [ ]:
vectors_body[0][5].shape

(256,)

In [ ]:
for i in range(len(sentences_y)):
    vectors_head.append([])
    temp = sentences_y[i]
    for j in range(len(temp)):
        vectors_head[i].append(head_embedding[temp[j]])

D:\Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [ ]:
vectors_head[0][5].shape

(38,)

## FastText

In [ ]:
from gensim.models import FastText

body_embedding_fasttext = FastText(sentences_X, size=100, window=5, min_count=1, workers=4,sg=1)
head_embedding_fasttext = FastText(sentences_y, size=100, window=5, min_count=1, workers=4,sg=1)

In [ ]:
body_embedding_fasttext.save('models/body_embedding_fasttext.model')
head_embedding_fasttext.save('models/head_embedding_fasttext.model')